# Feature extraction and Time Series Forecasting in kdb+

The purpose of this notebook is to demonstrate the use of feature extraction and selection in kdb+ as a tool for forecasting the future behaviour of a system. In this case we are using information about the historical behaviour of a stock with the goal of predicting the actual value of the stocks close price the next day.

As is demonstrated in the results section of this notebook this should **not** be used as a method for stock prediction and is shown here as a use-case to demonstrate the versatility of the FRESH algorithm, its use outside of classification and when dealing with historical data

### Loading Library Scripts and Data

In the following cell the functions contained in the ML-Toolkit including both the FRESH algorithm and broader utility functions are loaded into the notebook.

In [1]:
\l ../../ml.q
.ml.loadfile`:init.q

\c 15 100


~


Load in a dataset containing the daily Open/Low/High/Close/open interest prices and Volume of trades for Amazon Stock from 1997-2017

In [2]:
5#amzndaydata:{lower[cols x]xcol x}("DFFFFJJ";enlist ",")0:`:SampleDatasets/amzn_day.us.txt
-1!"This dataset contains stock information for ",(string count amzndaydata)," days."
5#amzndaydata:.ml.dropconstant[amzndaydata] / drop columns without variance 

date       open high low  close volume   openint
------------------------------------------------
1997.05.16 1.97 1.98 1.71 1.73  14700000 0      
1997.05.19 1.76 1.77 1.62 1.71  6106800  0      
1997.05.20 1.73 1.75 1.64 1.64  5467200  0      
1997.05.21 1.64 1.65 1.38 1.43  18853200 0      
1997.05.22 1.44 1.45 1.31 1.4   11776800 0      


"This dataset contains stock information for 5170 days."


date       open high low  close volume  
----------------------------------------
1997.05.16 1.97 1.98 1.71 1.73  14700000
1997.05.19 1.76 1.77 1.62 1.71  6106800 
1997.05.20 1.73 1.75 1.64 1.64  5467200 
1997.05.21 1.64 1.65 1.38 1.43  18853200
1997.05.22 1.44 1.45 1.31 1.4   11776800


## Assign extracted features and complete extraction

In this case we are attempting to use rolled table forecasting frames to predict the close price for the next day given extracted features from the previous 10 days. To generate the targets the first 10 days were omitted as these the rolled table frames in this case would be incomplete and as such may skew our results.

#### Set the target values for forecasting

In [3]:
tabletargets:10 _amzndaydata
targets:tabletargets[`close]

In [4]:
count targets

5160


#### Data preprocessing

Here we produce polynomial features from our data in order to allow for interactions between terms in the system to also be studied not just the individual features. The date column is also removed from the data as this is not used as a feature and will not be required given the data will be subject to a sliding window which negates its significance as a column.

In [5]:
/ Add 2nd order polynomial features to the table 
table:amzndaydata^.ml.polytab[flip 1_flip amzndaydata;2]
/ Remove the date column from the data as the rolling of data is independent of this
5#table:(1_cols t)#t:table 

open high low  close volume   open_high open_low open_close open_volume  high_low high_close high..
-------------------------------------------------------------------------------------------------..
1.97 1.98 1.71 1.73  14700000 3.9006    3.3687   3.4081     2.8959e+07   3.3858   3.4254     2.91..
1.76 1.77 1.62 1.71  6106800  3.1152    2.8512   3.0096     1.074797e+07 2.8674   3.0267     1.08..
1.73 1.75 1.64 1.64  5467200  3.0275    2.8372   2.8372     9458256      2.87     2.87       9567..
1.64 1.65 1.38 1.43  18853200 2.706     2.2632   2.3452     3.091925e+07 2.277    2.3595     3.11..
1.44 1.45 1.31 1.4   11776800 2.088     1.8864   2.016      1.695859e+07 1.8995   2.03       1.70..


---

### Producing a rolling table forecasting frame from the dataset

**The following function creates a rolling window with window lengths n on the data and implements feature extraction on each window. Caution should be exercised when applying this to large datasets as the number of calculations required will be;**
```q
(1+count table)-n
```
**It is here as an example of the use of rolling windows for feature forecasting**

In [6]:
rollcreatefeatures:{[x;fns;n] 
 raze{.ml.fresh.createfeatures[x;`placer;(-1)_cols x;y]}[;fns]each
 {update placer:last y from x y}[x;]each dropswin[n;til count x]}
dropswin:{(-1) _ (x-1) _ swin[x;y]}
swin:{[w;s]{1_x,y}\[w#0;s]}



---

## Feature Extraction

In [7]:
show dict:.ml.fresh.params

dict:update valid:0b from dict where f in `c3`numpeaks`quantile`fftcoeff`spktwelch`ratiobeyondsigma`augfuller

f             | pnum pnames pvals valid
--------------| -----------------------
absenergy     | 0                 1    
abssumchange  | 0                 1    
count         | 0                 1    
countabovemean| 0                 1    
countbelowmean| 0                 1    
firstmax      | 0                 1    
firstmin      | 0                 1    
hasdup        | 0                 1    
hasdupmax     | 0                 1    
hasdupmin     | 0                 1    
..


The following cell produces features extracted from 10 day sliding windows over the data, these are used in the prediction of the following days close price. 

The keyed column in the output table gives the column number of the last value in each window. 

All incomplete windows have been removed from the application of the sliding window and the  window for the last day is not created as this could not be used in the prediction of a target.

In [8]:
show tabraw:rollcreatefeatures[table;dict;10]
-1"The forecasting frame contains ",(string count tabraw)," datapoints.";

/home/mlteam/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


placer| open_absenergy open_abssumchange open_count open_countabovemean open_countbelowmean open_..
------| -----------------------------------------------------------------------------------------..
9     | 26.2488        0.89              10         5                   5                   0    ..
10    | 24.648         0.69              10         4                   6                   0    ..
11    | 23.8913        0.68              10         3                   7                   0    ..
12    | 23.0888        0.64              10         4                   6                   0    ..
13    | 22.4156        0.5               10         6                   4                   0.3  ..
14    | 22.6524        0.57              10         6                   4                   0.2  ..
15    | 23.4199        0.61              10         4                   6                   0.9  ..
16    | 24.0639        0.55              10         3                   7                   0.9  ..


## Feature significance  

In [14]:
show tabreduced:key[tabraw]!(.ml.fresh.significantfeatures[p;targets])#p:value tabraw
-1 "The number of columns in the initial dataset is: ",string count cols amzndaydata;
-1 "The number of columns in the unfiltered dataset is: ",string count cols tabraw;
-1 "The number of columns in the filtered dataset is: ",string count cols tabreduced;

placer| open_fftaggreg_skew open_fftaggreg_kurtosis high_fftaggreg_skew high_fftaggreg_kurtosis l..
------| -----------------------------------------------------------------------------------------..
9     | 3.34865             15.10603                3.338026            14.96626                4..
10    | 4.738168            28.64043                4.732788            28.36683                 ..
11    |                                                                                         4..
12    | 4.460971            23.85714                4.471995            23.85924                 ..
13    |                                                                                          ..
14    |                                             4.749852            26.05558                4..
15    | 4.738123            26.48774                                                            4..
16    |                                                                                          ..


## Convert Data

The data must now be converted to a matrix from a table in order to allow it to be passed to a machine learning algorithm for training.

In the below cell, I have reintroduced the original data for each day into the dataset such that it now contains both the original and derived datapoints

In [15]:
mattab:{flip value flip x}
rawdata:-1 _9 _amzndaydata
fitvalsfilter:0^mattab[rawdata,'value tabreduced]
fitvalsraw:0^mattab[rawdata,'value tabraw]
newpredictor:0^mattab[delete date from rawdata]

---

## Training the Model

Here we train our model using a gradient boost regressor to estimate the close price. The application of forms of regression on financial data can be quite tempermental with high dependency on how the data is split influencing the final result achieved.

In the definition of the gradient boosting regressor below the addition of``` `verbose pykw 1``` allows for the training times and training loss as a function of iterations to be displayed, this is omitted here as a means of ensuring the notebook is not cluttered

In [16]:
k:rand 1000
reg:.p.import[`sklearn.ensemble][`:GradientBoostingRegressor][`learning_rate pykw 0.1;`n_estimators pykw 200;`random_state pykw k]

### Fit data to the models and make predictions

In the below example:

* dict1 = input only the initial time series data
* dict2 = unfiltered data input.
* dict3 = filtered data input.

In [ ]:
dict1:.ml.traintestsplit[newpredictor;targets;0.2];
dict2:.ml.traintestsplit[fitvalsraw;targets;0.2];
dict3:.ml.traintestsplit[fitvalsfilter;targets;0.2];

reg[`:fit][dict1[`xtrain];dict1[`ytrain]]`;
pred1:reg[`:predict][dict1[`xtest]]`

reg[`:fit][dict2[`xtrain];dict2[`ytrain]]`;
pred2:reg[`:predict][dict2[`xtest]]`

reg[`:fit][dict3[`xtrain];dict3[`ytrain]]`;
pred3:reg[`:predict][dict3[`xtest]]`


## Results

Below the mean squared error is calculated between the predicted values of the next day close price and the actual value, this is a reasonable proxy for the accuracy of the model. The benchmark which we compare the model to is that todays close price is the best indicator of what tomorrows close will be.

In [ ]:
-1 "The mean square error(mse) between the test and prediction from the untreated data is: ",string .ml.mse[dict1[`ytest];pred1];
-1 "The mse between the test and predicted data without filtering is: ",string .ml.mse[dict2[`ytest];pred2];
-1 "The mse between the test and predicted data with filtering is: ",string .ml.mse[dict3[`ytest];pred3];
-1 "The mse between the current and previous day would be: ",string .ml.mse . 1 -1_\:amzndaydata[`close];

In [ ]:
plt:.p.import[`matplotlib]`:pyplot

plt[`:close][]`;
plt[`:figure][`figsize pykw (15 7.5)]`;
plt[`:title][`$"Next day close price prediction"];
plt[`:plot][dict2[`ytest];"r";`label pykw `$"real value"]`;
plt[`:plot][pred2;"k--";`label pykw `$"predicted value"]`;
plt[`:legend][]`;
plt[`:xlim][200;300]`;
plt[`:xlabel]["Random predicted date"];
plt[`:ylabel]["Close Price"];
plt[`:show][]`;


## Conclusions

Multiple runs of the above fit and results sections give an indication both of the usefulness of this form of analysis and the difficultly which is posed in the prediction of stock information. 

In the majority of cases it is seen that the mean squared error decreases in the order 

1. Untreated Data
2. Extracted Features without significance testing
3. Extracted Features with significance testing
4. Yesterday's price = Today's price

However depending on how the data is split by the traintestsplit function large variations in the the mean squared error can occur. This relates to the inclusion or omission of specific events which would cause large errors in prediction such as issues with the global economy or issues with staff. It can also have a detrimental effect on the order which is seen above with variations in this order possible if the features extracted from the split data are not truely indicative of the behaviour of the system.

---